In [1]:
import os
import io
import time
import traceback
import h5py
import tqdm
import copy

import pandas as pd
import numpy as np
from collections import defaultdict
from tifffile import imread
from skimage.metrics import normalized_root_mse, peak_signal_noise_ratio, structural_similarity
from math import isinf, isnan, sqrt
import hdf5plugin

In [2]:
PRESET = {
    0: 'NONE',
    10: 'ULTRAFAST',
    11: 'SUPERFAST',
    12: 'VERYFAST',
    13: 'FASTER',
    14: 'FAST',
    15: 'MEDIUM',
    16: 'SLOW',
    17: 'SLOWER',
    18: 'VERYSLOW',
    100: 'FASTEST',
    101: 'FASTER',
    102: 'FAST',
    103: 'MEDIUM',
    104: 'SLOW',
    105: 'SLOWER',
    106: 'VERYSLOW',
    200: 'ULTRAFAST',
    201: 'SUPERFAST',
    202: 'VERYFAST',
    203: 'FASTER',
    204: 'FAST',
    205: 'MEDIUM',
    206: 'SLOW',
    207: 'SLOWER',
    208: 'VERYSLOW',
    300: 'FASTEST',
    301: 'FASTER',
    302: 'FAST',
    303: 'MEDIUM',
    304: 'SLOW',
    305: 'SLOWER',
    306: 'VERYSLOW',
    400: 'ULTRAFAST',
    401: 'SUPERFAST',
    402: 'VERYFAST',
    403: 'MUCHFASTER',
    404: 'FASTER',
    405: 'FAST',
    406: 'LESSFAST',
    407: 'MEDIUM',
    408: 'LESSSLOW',
    409: 'SLOW',
    410: 'SLOWER',
    411: 'MUCHSLOWER',
    412: 'VERYSLOW',
    413: 'SUPERSLOW',
}

In [3]:
def preprocess_img(im, mode=None):
    if mode == '8bit':
        im = im.astype(float)
        im /= np.amax(im)
        im = np.power(im, 0.9) * 255
        im = np.array(im, dtype=np.uint8)
    if mode == '10bit':
        im = im.astype(float)
        im /= np.amax(im)
        im = np.power(im, 0.9) * 1023
        im = np.array(im, dtype=np.uint16)
        
    return im

In [4]:
def postprocess_img(im, mode=None, img_max=None):
    if mode == '8bit':
        im = im.astype(float)
        im /= np.amax(im)
        im = np.power(im, 1/0.9) * img_max
        im = np.array(im, dtype=np.uint16)
    if mode == '10bit':
        im = im.astype(float)
        im /= np.amax(im)
        im = np.power(im, 1/0.9) * img_max
        im = np.array(im, dtype=np.uint16)

    return im

In [5]:
# def preprocess_img(im, mode=None):
#     if im.dtype == np.uint16:
#         im = im.astype(np.float32)
#         if mode == '8bit':
#             im = np.power(im + 1, 0.5) - 1
#             im = np.array(im, dtype=np.uint8)
#         if mode == '10bit':
#             im = np.power(im + 1, 0.625) - 1
#             im = np.array(im, dtype=np.uint16)
        
#     return im

In [6]:
# def postprocess_img(im, mode=None):
#     if mode == '8bit':
#         im = im.astype(np.float32)
#         im = np.power(im + 1, 2) - 1
#         im = np.array(im, dtype=np.uint16)
#     if mode == '10bit':
#         im = im.astype(np.float32)
#         im = np.power(im + 1, 1.6) - 1
#         im = np.array(im, dtype=np.uint16)

#     return im

In [7]:
def test_compression(img, params={}, dataset=None, codec=None, cs=(256, 256, 256)):
    if '8bit' in codec:
        mode = '8bit'
    elif '10bit' in codec:
        mode = '10bit'
    elif 'JPEG' in codec:
        mode = '8bit'
    else:
        mode = None

    # if 'EM' in dataset:
    #     mode = None # no operation for EM

    try:
        if 'Blosc' in codec or 'Uncompressed' in codec:
            pre_img = copy.deepcopy(img)
        else:
            pre_img = preprocess_img(img, mode=mode)
        out = []
        if 'JPEG' in codec:
            filesize = []
            tmp = []
            enc_time = []
            dec_time = []
            for im in pre_img:
                with io.BytesIO() as tmpfile:
                    with h5py.File(tmpfile, 'w') as h5file:
                        start = time.time()
                        h5file.create_dataset('data', data=im, **params)
                        filesize.append(tmpfile.getbuffer().nbytes)
                        enc_time.append(time.time() - start)
                    with h5py.File(tmpfile, 'r') as h5file:
                        start = time.time()
                        tmp.append(np.array(h5file['data']))
                        dec_time.append(time.time() - start)

            tmp = np.stack(tmp, axis=0)
            out.append(sum(filesize))
            out.append(sum(enc_time))
            out.append(sum(dec_time))
        else:
            with io.BytesIO() as tmpfile:
                with h5py.File(tmpfile, 'w') as h5file:
                    start = time.time()
                    h5file.create_dataset('data', data=pre_img, **params, chunks=cs)
                    out.append(tmpfile.getbuffer().nbytes)
                    out.append(time.time() - start)
                with h5py.File(tmpfile, 'r') as h5file:
                    start = time.time()
                    tmp = np.array(h5file['data'])
                    out.append(time.time() - start)
                    
        if 'Blosc' in codec or 'Uncompressed' in codec:
            pass
        else:
            tmp = postprocess_img(tmp, mode, np.amax(img))
        # print(img.dtype, tmp.dtype)

        if dname == 'EM':
            tmp = tmp.astype(np.uint8)
    
        nrmse = normalized_root_mse(img, tmp)
        psnr = peak_signal_noise_ratio(img, tmp)
        ssim = structural_similarity(img, tmp, data_range=np.amax(img) - np.amin(img))
        out.append(nrmse)
        out.append(psnr)
        out.append(ssim)

    except Exception:
        print(traceback.format_exc())

    return out

In [8]:
data_path = '/data/duanb/'

In [9]:
datasets = {
    'Brainbow': np.reshape(np.transpose(imread(data_path + 'Brainbow/nTracer_sample.tif'), (1, 0, 2, 3)), (-1, 1024, 1024)),
    'NeuN': imread(data_path + 'NeuN/20211223_HDCF_R54-4_fibers11-16_hiRes_NeuN_histMatched_medianFilt.tif'),
    'EM': imread(data_path + 'EM/image.tif'),
    # 'fMOST': h5py.File('', 'r'),
    # 'MERFISH': np.reshape(np.transpose(imread(data_path + 'MERFISH/aligned_images0.tif'), (1, 0, 2, 3)), (-1, 2048, 2048)),
    'DRAQ5': h5py.File(data_path + 'DRAQ5/coord_-5.2,+18.6_2_1X.h5', 'r')['data'],
    'STORM': imread(data_path + 'STORM/Aquired STORM.tif'),
}

for k, v in datasets.items():
    print(k, v.shape, v.dtype)

Brainbow (260, 1024, 1024) uint16
NeuN (497, 1500, 3000) uint16
EM (500, 500, 500) uint8
DRAQ5 (1000, 2304, 2304) uint16
STORM (40000, 256, 256) uint16


In [10]:
# compression options
# blosc_settings = defaultdict(lambda: {})
# for ctype in ['lz4', 'lz4hc', 'zlib', 'zstd']:
#     for shuffle,shufflep in [('shuffle', 1), ('bitshuffle', 2)]:
#         for clevel in [3,5,8]:
#             key = f'BLOSC_{clevel}_{shuffle}_{ctype}'
#             toadd = {**blosc_settings[key], **hdf5plugin.Blosc(cname=ctype, clevel=clevel, shuffle=shufflep)}
#             blosc_settings[key] = toadd

# jpeg_settings = defaultdict(lambda: {})
# for pst in range(0, 101, 20):
#     key = f'JPEG_Quality{pst}'
#     param = (pst, 256, 256, 0)
#     toadd = {**jpeg_settings[key], **hdf5plugin.JPEG(*param)}
#     jpeg_settings[key] = toadd

x264_settings = defaultdict(lambda: {})
for pst in [10, 14, 15, 16]:
    for bit in [8, 10]:
        key = f'X264_{PRESET[pst]}_{bit}bit'
        if bit == 8:
            param = (2, 1, 256, 256, 256, 0, pst, 10, 10, 0, 0)
        else:
            param = (2, 1, 256, 256, 256, 1, pst, 10, 10, 0, 0)
        toadd = {**x264_settings[key], **hdf5plugin.FFMPEG(*param)}
        x264_settings[key] = toadd

x265_settings = defaultdict(lambda: {})
for pst in [200, 204, 205, 206]:
    for bit in [8, 10]:
        key = f'X265_{PRESET[pst]}_{bit}bit'
        if bit == 8:
            param = (4, 3, 256, 256, 256, 0, pst, 200, 15, 0, 0)
        else:
            param = (4, 3, 256, 256, 256, 1, pst, 200, 15, 0, 0)
        toadd = {**x265_settings[key], **hdf5plugin.FFMPEG(*param)}
        x265_settings[key] = toadd

av1_settings = defaultdict(lambda: {})
for pst in [403, 405, 407, 408]:
    for bit in [8, 10]:
        key = f'SVTAV1_{PRESET[pst]}_{bit}bit'
        if bit == 8:
            param = (6, 6, 256, 256, 256, 0, pst, 400, 20, 0, 0)
        else:
            param = (6, 6, 256, 256, 256, 1, pst, 400, 20, 0, 0)
        toadd = {**av1_settings[key], **hdf5plugin.FFMPEG(*param)}
        av1_settings[key] = toadd

compression_methods = {
    'Uncompressed':{},
    # **blosc_settings,
    # **jpeg_settings,
    **x264_settings,
    **x265_settings,
    **av1_settings,
}

In [11]:
N = 4
chunk_size = 256
subsampled_images = defaultdict(lambda: [])
for dname, dset in tqdm.tqdm(datasets.items()):
    for i in range(N):
        selector = None
        repeat = True
        while repeat:
            z = np.random.randint(0, max(1, dset.shape[0]-chunk_size))
            x = np.random.randint(0, max(1, dset.shape[1]-chunk_size))
            y = np.random.randint(0, max(1, dset.shape[2]-chunk_size))
            selector = [
                slice(z, z+chunk_size),
                slice(x, x+chunk_size),
                slice(y, y+chunk_size)
            ]
            if dset.shape[0] < chunk_size:
                selector[0] = slice(None)

            tmp = np.array(dset[tuple(selector)])

            if tmp.max() < 100:
                repeat = True
            else:
                repeat = False
                subsampled_images[dname].append(tmp)

100%|████████████████| 5/5 [00:01<00:00,  4.05it/s]


In [12]:
del datasets

In [13]:
data = pd.DataFrame({
    'Dataset': [],
    'Compression Method': [],
    'Compressed Size': [],
    'Read Time': [],
    'Write Time': [],
    'NRMSE': [],
    'PSNR': [],
    'SSIM': [],
    })

In [ ]:
benchmark_start = time.time()
last_start = benchmark_start

pbar = tqdm.tqdm(total=N*len(subsampled_images) * len(compression_methods), ncols=80, ascii=True)
for mname, mparam in compression_methods.items():
    for dname, dset in subsampled_images.items():
        for img_data in dset:
            # print(dname, mname)
            # if dname == 'EM' and '10bit' in mname: # no 10bit compression for already 8bit image
            #     pass
            # else:
            img = copy.deepcopy(img_data)
            s, wt, rt, nrmse, psnr, ssim = test_compression(img, params=mparam, dataset=dname, codec=mname)
            data = pd.concat([data, pd.DataFrame([{
                'Dataset': dname,
                'Compression Method': mname,
                'Compressed Size': s,
                'Read Time': rt,
                'Write Time': wt,
                'NRMSE': nrmse,
                'PSNR': psnr,
                'SSIM': ssim,
            }])], ignore_index=True)
            pbar.update(1)

print("Benchmark ended at", time.time() - benchmark_start)

  0%|                                                   | 0/500 [00:00<?, ?it/s]/home/duanb/mambaforge/envs/ffmpeg/lib/python3.11/site-packages/skimage/metrics/simple_metrics.py:163: RuntimeWarning: divide by zero encountered in scalar divide
  return 10 * np.log10((data_range ** 2) / err)
  1%|2                                        | 3/500 [00:45<2:05:10, 15.11s/it]

In [ ]:
means = data.groupby(['Dataset', 'Compression Method']).mean()
data['Compression Ratio'] = data.apply(
    lambda x: means.loc[x['Dataset']].loc['Uncompressed']['Compressed Size'] / x['Compressed Size'],
    axis=1
)

data['Effective Compression Rate (MB/s)'] = data.apply(
    lambda x: means.loc[x['Dataset']].loc['Uncompressed']['Compressed Size'] / (10**6) / x['Write Time'],
    axis=1
)

data['Effective Decompression Rate (MB/s)'] = data.apply(
    lambda x: means.loc[x['Dataset']].loc['Uncompressed']['Compressed Size'] / (10**6) / x['Read Time'],
    axis=1
)

data.to_csv('benchmark.csv')

In [ ]:
data.sort_values(by=['Compressed Size'], ascending=True).head(n=50)